# Data Management

# 1 How to Read the data

## 1.1 load Pandas and Numpy: data analysis tools.
    transform the csv to dataframe like excel.

In [30]:
import pandas as pd
import numpy as np

###  Read the csv file

Set the path and read the csv file

In [31]:
filename  ='F:\\Dropbox\\Flight\\Data\\Data_ontime\\On_Time_On_Time_Performance_2015_12.csv'

In [32]:
data = pd.read_csv(filename)

C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (48,76,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
df=data.sample(200)

In [34]:
df.to_csv(r"F:\Dropbox\2018\Nankai\201512_ontime.csv")

In [35]:
url = 'https://raw.githubusercontent.com/jukwan/nankai/master/201512_ontime.csv'

In [36]:
data = pd.read_csv(url)

In [37]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
163898,2015,4,12,10,4,2015-12-10,AA,19805,AA,N723UW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147434,2015,4,12,6,7,2015-12-06,AA,19805,AA,N483AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338939,2015,4,12,17,4,2015-12-17,UA,19977,UA,N38459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324541,2015,4,12,27,7,2015-12-27,UA,19977,UA,N817UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342408,2015,4,12,16,3,2015-12-16,UA,19977,UA,N414UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Take a quick look about data we load

In [38]:
data.shape 
# This shows the # of (rows, columns)

(200, 111)

In [39]:
data.head() # shows first 5 rows

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,162544,2015,4,12,5,6,2015-12-05,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,316322,2015,4,12,16,3,2015-12-16,OO,20304,OO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73457,2015,4,12,10,4,2015-12-10,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,124563,2015,4,12,24,4,2015-12-24,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,149188,2015,4,12,17,4,2015-12-17,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
data.tail()  # shows last 5 rows

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
195,56340,2015,4,12,28,1,2015-12-28,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,384969,2015,4,12,5,6,2015-12-05,WN,19393,WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,145524,2015,4,12,15,2,2015-12-15,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,183449,2015,4,12,27,7,2015-12-27,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,269593,2015,4,12,19,6,2015-12-19,OO,20304,OO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check the variables in data frame

In [41]:
data.columns.values

array(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth',
       'DayOfWeek', 'FlightDate', 'UniqueCarrier', 'AirlineID', 'Carrier',
       'TailNum', 'FlightNum', 'OriginAirportID', 'OriginAirportSeqID',
       'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState',
       'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID',
       'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac',
       'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff',
       'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay',
       'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk',
       'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime',
       'ActualElapsedTime', 'AirTime', 'Flights', 'Distance',
       'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay',

### Select necessary variables

In [42]:
 keyval=['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
           'UniqueCarrier', 'AirlineID', 'Carrier', 'TailNum',
            'FlightNum', 'Origin', 'OriginCityName', 
            'Dest', 'DestCityName',  
            'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15',
            'DepartureDelayGroups', 'DepTimeBlk',
            'TaxiOut', 'TaxiIn',
            'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
            'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode',
            'Diverted', 
            'AirTime', 'Flights', 'Distance', 'DistanceGroup',
            'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
            'DivAirportLandings',
            'DivReachedDest','DivActualElapsedTime', 'DivArrDelay', 'DivDistance',
            'Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn']

In [43]:
keyvals = [ 'DayOfWeek', 'FlightDate',
            'Carrier', 'TailNum',
            'FlightNum', 'Origin', 
            'Dest',   
            'DepTime', 'DepDelay', 'DepDelayMinutes', 
            'TaxiOut', 'TaxiIn',
            'ArrTime', 'ArrDelay', 'ArrDelayMinutes',
            'Cancelled',
            'Diverted', 
            'AirTime', 'Flights', 'Distance', 
            'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [44]:
df= data[keyvals] # make new dataframe which is subset of previous dataset.

In [45]:
df.describe()

,DayOfWeek,FlightNum,DepTime,DepDelay,DepDelayMinutes,TaxiOut,TaxiIn,ArrTime,ArrDelay,ArrDelayMinutes,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,200.000000,200.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,200.000000,200.0,198.000000,200.0,200.000000,42.000000,42.000000,42.000000,42.000000,42.000000
mean,3.620000,2248.595000,1295.424242,15.282828,17.621212,16.106061,7.085859,1409.429293,9.757576,16.919192,0.010000,0.0,116.974747,1.0,845.185000,34.952381,2.880952,8.571429,0.428571,27.880952
std,1.968496,1726.224562,528.040244,64.724412,64.003830,7.708118,4.220773,544.315770,67.405748,64.999676,0.099748,0.0,79.247056,0.0,655.938271,120.047136,9.409913,11.133651,2.777460,53.124429
min,1.000000,82.000000,2.000000,-15.000000,0.000000,5.000000,2.000000,6.000000,-43.000000,0.000000,0.000000,0.0,16.000000,1.0,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,851.500000,903.000000,-4.000000,0.000000,11.000000,4.000000,1018.500000,-12.000000,0.000000,0.000000,0.0,59.000000,1.0,368.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,1863.500000,1247.000000,-1.000000,0.000000,14.000000,6.000000,1417.000000,-5.000000,0.000000,0.000000,0.0,94.500000,1.0,656.000000,1.500000,0.000000,0.000000,0.000000,11.500000
75%,5.000000,3160.000000,1721.750000,10.000000,10.000000,19.000000,8.000000,1817.250000,9.000000,9.000000,0.000000,0.0,145.000000,1.0,1069.250000,19.750000,0.000000,16.000000,0.000000,33.750000
max,7.000000,7369.000000,2332.000000,768.000000,768.000000,55.000000,29.000000,2354.000000,797.000000,797.000000,1.000000,0.0,442.000000,1.0,4184.000000,768.000000,49.000000,35.000000,18.000000,309.000000


In [46]:
df['TaxiOut'].describe() # shows basic stats for 1 variable

count    198.000000
mean      16.106061
std        7.708118
min        5.000000
25%       11.000000
50%       14.000000
75%       19.000000
max       55.000000
Name: TaxiOut, dtype: float64

In [47]:
df[['TaxiOut','TaxiIn']].describe() # more than one.

,TaxiOut,TaxiIn
count,198.000000,198.000000
mean,16.106061,7.085859
std,7.708118,4.220773
min,5.000000,2.000000
25%,11.000000,4.000000
50%,14.000000,6.000000
75%,19.000000,8.000000
max,55.000000,29.000000


### Check the missing values

In [48]:
df.isnull().sum() 

DayOfWeek              0
FlightDate             0
Carrier                0
TailNum                0
FlightNum              0
Origin                 0
Dest                   0
DepTime                2
DepDelay               2
DepDelayMinutes        2
TaxiOut                2
TaxiIn                 2
ArrTime                2
ArrDelay               2
ArrDelayMinutes        2
Cancelled              0
Diverted               0
AirTime                2
Flights                0
Distance               0
CarrierDelay         158
WeatherDelay         158
NASDelay             158
SecurityDelay        158
LateAircraftDelay    158
dtype: int64

### Counting unique values in specific variables

In [49]:
df.Cancelled.value_counts()

0.0    198
1.0      2
Name: Cancelled, dtype: int64

In [50]:
df['Diverted'].value_counts()

0.0    200
Name: Diverted, dtype: int64

In [51]:
8063+1450

9513

In [52]:
df['Carrier'].unique()

array(['AA', 'OO', 'EV', 'WN', 'HA', 'DL', 'UA', 'MQ', 'F9', 'AS', 'NK',
       'B6', 'VX'], dtype=object)

## 2. Sort the data
    Sort by multiple column values

In [53]:
df.sort_values(by =['Carrier','DayOfWeek','Origin','Dest'],  ascending=True)

,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,...,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
50,1,2015-12-21,AA,N951UW,899,BOS,PHL,1824.0,-6.0,0.0,...,0.0,0.0,77.0,1.0,280.0,NaN,NaN,NaN,NaN,NaN
143,1,2015-12-28,AA,N805AW,1763,CLT,ORF,1849.0,49.0,49.0,...,0.0,0.0,45.0,1.0,290.0,40.0,0.0,0.0,0.0,5.0
98,1,2015-12-21,AA,N4WKAA,2214,DFW,ABQ,856.0,-4.0,0.0,...,0.0,0.0,91.0,1.0,569.0,NaN,NaN,NaN,NaN,NaN
66,1,2015-12-21,AA,N342AA,942,JFK,MIA,1113.0,-2.0,0.0,...,0.0,0.0,153.0,1.0,1089.0,NaN,NaN,NaN,NaN,NaN
150,1,2015-12-14,AA,N3EBAA,1375,LAS,LAX,1009.0,-6.0,0.0,...,0.0,0.0,45.0,1.0,236.0,NaN,NaN,NaN,NaN,NaN
60,1,2015-12-07,AA,N475AA,1046,MCI,DFW,1059.0,-4.0,0.0,...,0.0,0.0,71.0,1.0,460.0,NaN,NaN,NaN,NaN,NaN
197,2,2015-12-15,AA,N3BBAA,2187,BDL,DFW,607.0,-3.0,0.0,...,0.0,0.0,232.0,1.0,1471.0,NaN,NaN,NaN,NaN,NaN
100,2,2015-12-22,AA,N524UW,708,CLT,LAS,1825.0,10.0,10.0,...,0.0,0.0,280.0,1.0,1916.0,0.0,0.0,15.0,0.0,10.0
157,2,2015-12-29,AA,N475AA,2303,DEN,ORD,1726.0,6.0,6.0,...,0.0,0.0,119.0,1.0,888.0,NaN,NaN,NaN,NaN,NaN
47,2,2015-12-22,AA,N552UW,518,DEN,PHL,45.0,0.0,0.0,...,0.0,0.0,171.0,1.0,1558.0,NaN,NaN,NaN,NaN,NaN


In [54]:
df.sort_values(by='DepDelay', ascending=False, na_position='first')

,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,...,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
68,1,2015-12-28,WN,N762SW,706,ATL,MDW,NaN,NaN,NaN,...,1.0,0.0,NaN,1.0,591.0,NaN,NaN,NaN,NaN,NaN
126,1,2015-12-28,WN,N709SW,119,MCI,STL,NaN,NaN,NaN,...,1.0,0.0,NaN,1.0,237.0,NaN,NaN,NaN,NaN,NaN
13,7,2015-12-27,AA,N3HXAA,1338,SEA,MIA,1003.0,768.0,768.0,...,0.0,0.0,334.0,1.0,2724.0,768.0,0.0,29.0,0.0,0.0
82,7,2015-12-27,UA,N492UA,1948,IAH,PHX,2332.0,325.0,325.0,...,0.0,0.0,145.0,1.0,1009.0,0.0,0.0,6.0,0.0,309.0
181,3,2015-12-30,DL,N959DN,1858,ATL,AUS,1223.0,143.0,143.0,...,0.0,0.0,127.0,1.0,813.0,0.0,49.0,0.0,0.0,89.0
19,3,2015-12-23,UA,N18243,1131,IAD,SJU,12.0,132.0,132.0,...,0.0,0.0,207.0,1.0,1571.0,132.0,0.0,1.0,0.0,0.0
164,2,2015-12-22,OO,N608SK,4725,SUN,SLC,2005.0,126.0,126.0,...,0.0,0.0,37.0,1.0,222.0,0.0,0.0,0.0,0.0,112.0
114,5,2015-12-18,VX,N837VA,420,LAX,JFK,116.0,126.0,126.0,...,0.0,0.0,265.0,1.0,2475.0,0.0,0.0,0.0,0.0,102.0
80,2,2015-12-29,MQ,N525MQ,3532,OKC,DFW,854.0,118.0,118.0,...,0.0,0.0,38.0,1.0,175.0,116.0,0.0,0.0,0.0,0.0
39,3,2015-12-09,AA,N558UW,603,MCO,PHX,1902.0,107.0,107.0,...,0.0,0.0,227.0,1.0,1849.0,65.0,0.0,0.0,0.0,0.0


### Group by 

In [55]:
df.groupby(['Origin'])['ArrDelay'].describe()

,count,mean,std,min,25%,50%,75%,max
Origin,,,,,,,,
ABQ,3.0,19.666667,42.910760,-9.0,-5.00,-1.0,34.00,69.0
APN,1.0,-13.000000,NaN,-13.0,-13.00,-13.0,-13.00,-13.0
ATL,15.0,7.933333,42.690693,-27.0,-18.50,-2.0,8.00,138.0
AUS,6.0,-10.000000,18.231840,-24.0,-19.75,-14.5,-12.25,26.0
AZO,1.0,-18.000000,NaN,-18.0,-18.00,-18.0,-18.00,-18.0
BDL,2.0,-6.000000,9.899495,-13.0,-9.50,-6.0,-2.50,1.0
BHM,2.0,5.000000,18.384776,-8.0,-1.50,5.0,11.50,18.0
BIS,2.0,2.000000,11.313708,-6.0,-2.00,2.0,6.00,10.0
BNA,2.0,13.000000,31.112698,-9.0,2.00,13.0,24.00,35.0


In [56]:
df.groupby(['Carrier'])['DepDelay','ArrDelay'].mean()

## Mean can replaced by those functions
# Function	Description
# mean()	Compute mean of groups
# sum()	Compute sum of group values
# size()	Compute group sizes
# count()	Compute count of group
# std()	Standard deviation of groups
# var()	Compute variance of groups
# sem()	Standard error of the mean of groups
# describe()	Generates descriptive statistics
# first()	Compute first of group values
# last()	Compute last of group values
# nth()	Take nth value, or a subset if n is a list
# min()	Compute min of group values
# max()	Compute max of group values

,DepDelay,ArrDelay
Carrier,,
AA,29.081081,27.135135
AS,-9.000000,-16.000000
B6,-1.666667,-11.000000
DL,12.266667,4.800000
EV,2.411765,-0.588235
F9,6.250000,2.500000
HA,-6.000000,-8.333333
MQ,16.750000,5.750000
NK,4.800000,8.800000


In [57]:
df.groupby(['Carrier']).get_group('DL').mean()

DayOfWeek               3.500000
FlightNum            1647.733333
DepTime              1421.233333
DepDelay               12.266667
DepDelayMinutes        14.500000
TaxiOut                20.700000
TaxiIn                  6.966667
ArrTime              1679.766667
ArrDelay                4.800000
ArrDelayMinutes        14.600000
Cancelled               0.000000
Diverted                0.000000
AirTime               129.533333
Flights                 1.000000
Distance              938.933333
CarrierDelay            9.666667
WeatherDelay           13.333333
NASDelay               14.500000
SecurityDelay           0.000000
LateAircraftDelay      33.333333
dtype: float64

In [58]:
a= df.groupby(['Origin','Dest'])['DepDelay','ArrDelay']


In [59]:
a.mean()

DepDelay  ArrDelay
Origin Dest                    
ABQ    DEN       -3.0      -1.0
       MDW       -3.0      -9.0
       PHX       65.0      69.0
APN    DTW      -12.0     -13.0
ATL    AGS       -4.0      -2.0
       AUS      143.0     138.0
       BOS       21.0       3.0
       DFW        7.0       6.0
       EWR       22.5      10.5
       FSD       -6.0       4.0
       HPN       -6.0     -21.0
       MDW        NaN       NaN
       MSP       -5.0     -27.0
       MSY        9.0      10.0
       ORD       -1.0     -23.0
       RIC       -3.0      -7.0
       ROC       -2.0     -24.0
       TUS       32.0      57.0
       VLD       -9.0     -16.0
AUS    ATL       -5.0     -16.0
       DEN       -4.0     -24.0
       DFW       -2.0     -12.0
       ELP       32.0      26.0
       MCO       -1.0     -13.0
       PDX      -13.0     -21.0
AZO    ORD       -6.0     -18.0
BDL    ATL       -3.0     -13.0
       DFW       -3.0       1.0
BHM    ATL       -7.0      -8.0
       MIA       18.0      18.0
...               ...       ...
PHL    ATL       -1.0       5.0
       DTW        1.0       1.0
PHX    ATL        2.0      -3.0
       CMH        0.0       5.0
       DAL       -1.0      16.0
       MCO        0.0      -4.0
       SFO        3.0      10.0
       TUL       -2.0      -3.0
RSW    ACY       -9.0     -14.0
SAN    LAS        0.0     -12.0
       SMF        6.0       3.0
SEA    BOI      -15.0     -23.0
       DFW       -2.0      16.0
       MIA      768.0     797.0
SFO    BWI       -3.0      -9.0
SGF    ORD       12.0       7.0
SJC    SAN       10.0       6.0
SJU    FLL        8.0      -7.0
SLC    BWI       -3.0      -5.0
       LAS       -4.0      -6.0
SMF    DFW       -2.0     -12.0
STL    MSY       -2.5       5.0
       PHX       40.0      63.0
SUN    SLC      126.0     112.0
TPA    ATL       19.0      16.0
       RDU        0.0      -6.0
TTN    ORD       -1.0      -1.0
TUL    DFW       -7.0     -10.0
XNA    ORD      -12.0     -34.0
YUM    PHX      -10.0     -26.0

[190 rows x 2 columns]

# 3. Variable Management

### Drop the variables

In [60]:
data.drop(['Div1WheelsOn'], axis=1)

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,162544,2015,4,12,5,6,2015-12-05,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,316322,2015,4,12,16,3,2015-12-16,OO,20304,OO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73457,2015,4,12,10,4,2015-12-10,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,124563,2015,4,12,24,4,2015-12-24,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,149188,2015,4,12,17,4,2015-12-17,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,413828,2015,4,12,15,2,2015-12-15,WN,19393,WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,364197,2015,4,12,11,5,2015-12-11,OO,20304,OO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,418622,2015,4,12,15,2,2015-12-15,WN,19393,WN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,115709,2015,4,12,13,7,2015-12-13,HA,19690,HA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,161877,2015,4,12,6,7,2015-12-06,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop the row by column values

In [61]:
df.query("Origin =='BOS' or Origin =='ATL'")

,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,...,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
3,4,2015-12-24,AA,N3JBAA,205,BOS,LAX,635.0,12.0,12.0,...,0.0,0.0,409.0,1.0,2611.0,0.0,12.0,34.0,0.0,0.0
5,2,2015-12-15,WN,N638SW,4042,ATL,BOS,826.0,21.0,21.0,...,0.0,0.0,115.0,1.0,946.0,NaN,NaN,NaN,NaN,NaN
34,4,2015-12-17,DL,N980AT,2042,ATL,EWR,1439.0,45.0,45.0,...,0.0,0.0,97.0,1.0,746.0,0.0,0.0,35.0,0.0,0.0
42,2,2015-12-29,DL,N949DL,1901,BOS,ATL,1402.0,107.0,107.0,...,0.0,0.0,147.0,1.0,946.0,51.0,23.0,19.0,0.0,33.0
46,5,2015-12-04,DL,N689DL,1978,ATL,RIC,2203.0,-3.0,0.0,...,0.0,0.0,62.0,1.0,481.0,NaN,NaN,NaN,NaN,NaN
50,1,2015-12-21,AA,N951UW,899,BOS,PHL,1824.0,-6.0,0.0,...,0.0,0.0,77.0,1.0,280.0,NaN,NaN,NaN,NaN,NaN
68,1,2015-12-28,WN,N762SW,706,ATL,MDW,NaN,NaN,NaN,...,1.0,0.0,NaN,1.0,591.0,NaN,NaN,NaN,NaN,NaN
71,3,2015-12-09,B6,N216JB,1081,BOS,RIC,929.0,-1.0,0.0,...,0.0,0.0,79.0,1.0,474.0,NaN,NaN,NaN,NaN,NaN
76,3,2015-12-23,DL,N906DA,1739,ATL,TUS,1952.0,32.0,32.0,...,0.0,0.0,224.0,1.0,1541.0,0.0,8.0,25.0,0.0,24.0
93,4,2015-12-10,DL,N982AT,2012,ATL,ORD,724.0,-1.0,0.0,...,0.0,0.0,90.0,1.0,606.0,NaN,NaN,NaN,NaN,NaN


In [62]:
df[df['DepDelay']>0]

,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,...,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
3,4,2015-12-24,AA,N3JBAA,205,BOS,LAX,635.0,12.0,12.0,...,0.0,0.0,409.0,1.0,2611.0,0.0,12.0,34.0,0.0,0.0
5,2,2015-12-15,WN,N638SW,4042,ATL,BOS,826.0,21.0,21.0,...,0.0,0.0,115.0,1.0,946.0,NaN,NaN,NaN,NaN,NaN
7,2,2015-12-15,WN,N8302F,286,OMA,LAS,1429.0,74.0,74.0,...,0.0,0.0,145.0,1.0,1099.0,20.0,0.0,0.0,0.0,34.0
13,7,2015-12-27,AA,N3HXAA,1338,SEA,MIA,1003.0,768.0,768.0,...,0.0,0.0,334.0,1.0,2724.0,768.0,0.0,29.0,0.0,0.0
17,5,2015-12-04,UA,N76523,548,MCO,DEN,1223.0,6.0,6.0,...,0.0,0.0,213.0,1.0,1546.0,NaN,NaN,NaN,NaN,NaN
18,4,2015-12-31,DL,N895AT,858,PBI,JFK,1310.0,55.0,55.0,...,0.0,0.0,125.0,1.0,1028.0,6.0,0.0,0.0,0.0,39.0
19,3,2015-12-23,UA,N18243,1131,IAD,SJU,12.0,132.0,132.0,...,0.0,0.0,207.0,1.0,1571.0,132.0,0.0,1.0,0.0,0.0
21,4,2015-12-10,UA,N494UA,372,PHX,SFO,829.0,3.0,3.0,...,0.0,0.0,106.0,1.0,651.0,NaN,NaN,NaN,NaN,NaN
22,2,2015-12-15,WN,N767SW,1214,SJU,FLL,738.0,8.0,8.0,...,0.0,0.0,134.0,1.0,1046.0,NaN,NaN,NaN,NaN,NaN
26,1,2015-12-28,EV,N905EV,2848,DFW,GRK,1156.0,66.0,66.0,...,0.0,0.0,35.0,1.0,134.0,24.0,0.0,11.0,0.0,42.0


In [63]:
df.query('DepDelay > 0 and ArrDelay ==0')

,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,...,Cancelled,Diverted,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
144,2,2015-12-01,WN,N733SA,440,MDW,DSM,2158.0,3.0,3.0,...,0.0,0.0,54.0,1.0,306.0,NaN,NaN,NaN,NaN,NaN
157,2,2015-12-29,AA,N475AA,2303,DEN,ORD,1726.0,6.0,6.0,...,0.0,0.0,119.0,1.0,888.0,NaN,NaN,NaN,NaN,NaN


### Creating Dummy Variables

In [64]:
pd.get_dummies(df, columns=["Carrier"])

,DayOfWeek,FlightDate,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,TaxiOut,...,Carrier_DL,Carrier_EV,Carrier_F9,Carrier_HA,Carrier_MQ,Carrier_NK,Carrier_OO,Carrier_UA,Carrier_VX,Carrier_WN
0,6,2015-12-05,N128UW,1829,CLT,TPA,1752.0,-3.0,0.0,14.0,...,0,0,0,0,0,0,0,0,0,0
1,3,2015-12-16,N685BR,7369,APN,DTW,1003.0,-12.0,0.0,14.0,...,0,0,0,0,0,0,1,0,0,0
2,4,2015-12-10,N16571,4137,ORD,AZO,1542.0,-8.0,0.0,10.0,...,0,1,0,0,0,0,0,0,0,0
3,4,2015-12-24,N3JBAA,205,BOS,LAX,635.0,12.0,12.0,21.0,...,0,0,0,0,0,0,0,0,0,0
4,4,2015-12-17,N3EJAA,2389,DFW,SNA,1522.0,-3.0,0.0,13.0,...,0,0,0,0,0,0,0,0,0,0
5,2,2015-12-15,N638SW,4042,ATL,BOS,826.0,21.0,21.0,10.0,...,0,0,0,0,0,0,0,0,0,1
6,5,2015-12-11,N774SK,5172,FAT,LAX,1312.0,-4.0,0.0,29.0,...,0,0,0,0,0,0,1,0,0,0
7,2,2015-12-15,N8302F,286,OMA,LAS,1429.0,74.0,74.0,10.0,...,0,0,0,0,0,0,0,0,0,1
8,7,2015-12-13,N488HA,343,HNL,LIH,1202.0,-8.0,0.0,15.0,...,0,0,0,1,0,0,0,0,0,0
9,7,2015-12-06,N760US,665,CMH,LAX,834.0,-11.0,0.0,28.0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
s = pd.get_dummies(df.Carrier)

In [66]:
df=df.join(s)

In [33]:
df.head()

,DayOfWeek,FlightDate,Carrier,TailNum,FlightNum,Origin,Dest,DepTime,DepDelay,DepDelayMinutes,...,DL,EV,F9,HA,MQ,NK,OO,UA,VX,WN
0,5,2015-12-04,DL,N344NW,1185,SLC,EWR,956.0,-9.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,5,2015-12-04,DL,N344NW,1185,EWR,SLC,1700.0,-10.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,5,2015-12-04,DL,N959DN,1186,ATL,LGA,1005.0,20.0,20.0,...,1,0,0,0,0,0,0,0,0,0
3,5,2015-12-04,DL,N342NB,1187,BOS,JFK,559.0,-1.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,5,2015-12-04,DL,N332NB,1188,MSP,PHL,1008.0,-8.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [34]:
## give id to airport
df['airs'] = df['Carrier'].astype('category')
df['airs'].dtypes
df['airs_cat']= df['airs'].cat.codes
df['airs_cat'].value_counts()

12    107476
0      76562
3      70639
9      47590
4      44140
10     43443
2      23343
7      20993
1      14459
8      10671
5       8120
6       6260
11      5534
Name: airs_cat, dtype: int64

### give market id 
market combination of departure and arrival airport

In [29]:
df['marketId']=df.groupby(['Origin','Dest']).ngroup().add(1)
df.groupby(['Origin','Dest'])['DepDelay','ArrDelay','marketId'].mean()

DepDelay  ArrDelay  marketId
Origin Dest                              
ABQ    DEN       -3.0      -1.0         1
       MDW       -3.0      -9.0         2
       PHX       65.0      69.0         3
APN    DTW      -12.0     -13.0         4
ATL    AGS       -4.0      -2.0         5
       AUS      143.0     138.0         6
       BOS       21.0       3.0         7
       DFW        7.0       6.0         8
       EWR       22.5      10.5         9
       FSD       -6.0       4.0        10
       HPN       -6.0     -21.0        11
       MDW        NaN       NaN        12
       MSP       -5.0     -27.0        13
       MSY        9.0      10.0        14
       ORD       -1.0     -23.0        15
       RIC       -3.0      -7.0        16
       ROC       -2.0     -24.0        17
       TUS       32.0      57.0        18
       VLD       -9.0     -16.0        19
AUS    ATL       -5.0     -16.0        20
       DEN       -4.0     -24.0        21
       DFW       -2.0     -12.0        22
       ELP       32.0      26.0        23
       MCO       -1.0     -13.0        24
       PDX      -13.0     -21.0        25
AZO    ORD       -6.0     -18.0        26
BDL    ATL       -3.0     -13.0        27
       DFW       -3.0       1.0        28
BHM    ATL       -7.0      -8.0        29
       MIA       18.0      18.0        30
...               ...       ...       ...
PHL    ATL       -1.0       5.0       161
       DTW        1.0       1.0       162
PHX    ATL        2.0      -3.0       163
       CMH        0.0       5.0       164
       DAL       -1.0      16.0       165
       MCO        0.0      -4.0       166
       SFO        3.0      10.0       167
       TUL       -2.0      -3.0       168
RSW    ACY       -9.0     -14.0       169
SAN    LAS        0.0     -12.0       170
       SMF        6.0       3.0       171
SEA    BOI      -15.0     -23.0       172
       DFW       -2.0      16.0       173
       MIA      768.0     797.0       174
SFO    BWI       -3.0      -9.0       175
SGF    ORD       12.0       7.0       176
SJC    SAN       10.0       6.0       177
SJU    FLL        8.0      -7.0       178
SLC    BWI       -3.0      -5.0       179
       LAS       -4.0      -6.0       180
SMF    DFW       -2.0     -12.0       181
STL    MSY       -2.5       5.0       182
       PHX       40.0      63.0       183
SUN    SLC      126.0     112.0       184
TPA    ATL       19.0      16.0       185
       RDU        0.0      -6.0       186
TTN    ORD       -1.0      -1.0       187
TUL    DFW       -7.0     -10.0       188
XNA    ORD      -12.0     -34.0       189
YUM    PHX      -10.0     -26.0       190

[190 rows x 3 columns]